# Analytics
> Investigating voting behavior.

In [1]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#hide
import sys
sys.path.append('..')

In [6]:
import pandas as pd
from bundestag import similarity

import numpy as np

In [7]:
%%time
df = pd.read_parquet('../votes.parquet')

CPU times: user 93.8 ms, sys: 172 ms, total: 266 ms
Wall time: 211 ms


In [8]:
%%time
other_cols = ['date', 'Fraktion/Gruppe']
df_squished = similarity.get_squished_dataframe(df, other_cols=other_cols)

CPU times: user 453 ms, sys: 203 ms, total: 656 ms
Wall time: 662 ms


In [9]:
df_squished.head()

,Bezeichnung,issue,date,Fraktion/Gruppe,vote
0,Stephan Albani,2015-07-02 Subventionen für britisches Atomwe...,2015-07-02,CDU/CSU,ja
1,Katrin Albsteiger,2015-07-02 Subventionen für britisches Atomwe...,2015-07-02,CDU/CSU,ja
2,Peter Altmaier,2015-07-02 Subventionen für britisches Atomwe...,2015-07-02,CDU/CSU,nichtabgegeben
3,Artur Auernhammer,2015-07-02 Subventionen für britisches Atomwe...,2015-07-02,CDU/CSU,ja
4,Dorothee Bär,2015-07-02 Subventionen für britisches Atomwe...,2015-07-02,CDU/CSU,ja


In [12]:
appearances = df_squished['Bezeichnung'].value_counts()

In [14]:
display(appearances.head(20), appearances.tail(20))

Christine Buchholz                  425
Thomas Nord                         425
Caren Lay                           425
Dr. Peter Tauber                    425
Ulla Schmidt (Aachen)               425
Jutta Krellmann                     425
Axel E. Fischer (Karlsruhe-Land)    425
Johannes Röring                     425
Eckhard Pols                        425
Dorothee Bär                        425
Michael Roth (Heringen)             425
Florian Pronold                     425
Anita Schäfer (Saalstadt)           425
Christian Hirte                     425
Dr. Kirsten Tackmann                425
Hans-Georg von der Marwitz          425
Caren Marks                         425
Alois Karl                          425
Dagmar Freitag                      425
Stephan Mayer (Altötting)           425
Name: Bezeichnung, dtype: int64

Dr. h.c. Susanne Kastner              10
Reiner Deutschmann                    10
Gerhard Johann Drexler                10
Dr. h.c. Wolfgang Thierse             10
Siegfried Kauder (Villingen-Schw.)    10
Dr. h.c. Jürgen Koppelin              10
Gerhard Drexler                        9
HonD Albert Weiler                     8
Christian Ahrendt                      6
Fritz Kuhn                             6
Dr. h. c. Albert Weiler                5
Dr. Hermann Ott                        5
Silke Launert                          5
Frhr. Von Christian Stetten            4
Marion Marga Herdan                    3
Reinhold Jost                          2
Julia Verlinden                        2
Susanne Kickbusch                      1
Alexander S. Neu                       1
Dr. Dr. h.c. Bernd Fabritius           1
Name: Bezeichnung, dtype: int64

In [15]:
df_squished.describe()

<ipython-input-15-502a8846bc9f>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_squished.describe()


,Bezeichnung,issue,date,Fraktion/Gruppe,vote
count,281053,281053,281053,281053,281053
unique,1234,425,166,11,4
top,Christine Buchholz,2020-05-14 Bereitstellung des ESM-Instruments...,2017-06-01 00:00:00,CDU/CSU,ja
freq,425,709,8190,117592,148186
first,NaN,NaN,2012-12-13 00:00:00,NaN,NaN
last,NaN,NaN,2020-09-10 00:00:00,NaN,NaN


In [17]:
df_squished['Fraktion/Gruppe'].value_counts()

CDU/CSU                  117592
SPD                       72955
DIE LINKE.                25805
BÜ90/GR                   25186
FDP                       17833
AfD                       15749
DIE LINKE                  2731
BÜNDNIS`90/DIE GRÜNEN      2496
Fraktionslos                662
fraktionslose                32
fraktionslos                 12
Name: Fraktion/Gruppe, dtype: int64

In [19]:
df_squished['vote'].value_counts(normalize=True)

ja                0.527253
nein              0.347949
nichtabgegeben    0.092232
Enthaltung        0.032567
Name: vote, dtype: float64

In [22]:
df_squished.loc[df_squished['vote']=='nichtabgegeben', 'Bezeichnung'].value_counts()

Dr. Angela Merkel        306
Sigmar Gabriel           257
Peter Altmaier           238
Dr. Sahra Wagenknecht    213
Dr. Gregor Gysi          183
                        ... 
Helga Daub                 1
Doris  Barnett             1
Heiner Kamp                1
Michael Kauch              1
Hans-Werner Kammer         1
Name: Bezeichnung, Length: 1123, dtype: int64

In [46]:
def add_share(df:pd.DataFrame):
    df.loc[:,'share'] = df['count'] / df['count'].groupby(level='Bezeichnung').sum()
    assert np.allclose(df['share'].groupby(level='Bezeichnung').sum(), 1)
    return df

vote_stats = (df_squished.groupby(['Bezeichnung', 'vote'])
              .size()
              .to_frame()
              .rename(columns={0:'count'})
              .pipe(add_share))
vote_stats.head()

count     share
Bezeichnung         vote                           
Achim Post (Minden) Enthaltung          1  0.002618
                    ja                234  0.612565
                    nein              111  0.290576
                    nichtabgegeben     36  0.094241
Agnes Alpers        Enthaltung          8  0.068966

In [48]:
vote_stats.loc[pd.IndexSlice[:,'nichtabgegeben'],:].sort_values('share', ascending=False).head(20)

,,count,share
Bezeichnung,vote,,
Ottmar Schreiner,nichtabgegeben,16,0.941176
Silvia Schmidt (Eisleben),nichtabgegeben,39,0.906977
Annette Schavan,nichtabgegeben,19,0.863636
Kornelia Möller,nichtabgegeben,36,0.837209
Fritz Kuhn,nichtabgegeben,5,0.833333
Jimmy Schulz,nichtabgegeben,133,0.764368
Sahra Wagenknecht,nichtabgegeben,31,0.720930
Dr. Angela Merkel,nichtabgegeben,306,0.720000
Sigmar Gabriel,nichtabgegeben,257,0.692722


In [54]:
vote_stats.loc[vote_stats['count']>25].loc[pd.IndexSlice[:,'nichtabgegeben'],:].sort_values('share', ascending=True).head(20)

,,count,share
Bezeichnung,vote,,
Dr. Kirsten Tackmann,nichtabgegeben,26,0.061176
Olav Gutting,nichtabgegeben,26,0.061176
Mechthild Heil,nichtabgegeben,26,0.061176
Tabea Rößner,nichtabgegeben,26,0.061176
Marcus Weinberg (Hamburg),nichtabgegeben,26,0.061176
Andreas G. Lämmel,nichtabgegeben,26,0.061176
Markus Kurth,nichtabgegeben,26,0.061176
Stefan Liebich,nichtabgegeben,27,0.063529
Dr. Konstantin von Notz,nichtabgegeben,27,0.063529


In [59]:
import plotly.express as px

In [62]:
fig = px.line(df_squished.groupby(['date'])['Bezeichnung'].nunique().to_frame().reset_index(), x='date', y='Bezeichnung')
fig.update_yaxes(range=(0,720))

In [64]:
fig = px.scatter(df_squished.groupby(['date','vote'])['Bezeichnung'].nunique().to_frame().reset_index(), x='date', y='Bezeichnung', color='vote')
fig.update_yaxes(range=(0,720))

In [71]:
fig = px.scatter(df_squished.groupby(['date','vote', 'Fraktion/Gruppe'])['Bezeichnung'].nunique().to_frame().reset_index(), x='date', y='Bezeichnung', color='vote',
                 facet_row='Fraktion/Gruppe', height=2000)
fig.update_yaxes(range=(0,720))

party agreement vs time